In [8]:
import sqlite3
import pandas as pd
import numpy as np
import time
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.moses import MosesDetokenizer

from stop_words import get_stop_words
from collections import defaultdict
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import TaggedDocument,TaggedLineDocument
from gensim.models import Doc2Vec
import gensim.models.doc2vec



# #from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
# from collections import OrderedDict
# import multiprocessing
# from IPython.display import Image
# import requests
# from random import randint 
# from sklearn.manifold import TSNE
# from sklearn.cluster import AffinityPropagation
# #import matplotlib.pyplot as plt
# #%matplotlib inline

In [25]:
conn = sqlite3.connect("volume2/testDB.db")

##### saving abstract tokens to db

In [26]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('PLOS_ALL',), ('PLOS_ALL_tok',), ('ed_concat_abtoks',), ('ed_concat',)]


In [27]:
cursor.execute('drop table if exists ed_concat_abtoks')

In [28]:
df = pd.read_sql('SELECT * from ed_concat', conn)

In [29]:
df.head()

,index,editors,ab_toks
0,0,Alash'le G. Abimiku,"'objectiveto', 'evalu', 'cost', 'effect', 'qua..."
1,1,Alessandro D'Ausilio,"'test', 'whether', 'pre', 'assign', 'arm', 'mo..."
2,2,Anderson de Souza Sant'Ana,"'human', 'migrat', 'impli', 'adapt', 'new', 'e..."
3,3,Avi Ma'ayan,"'real', 'time', 'quantit', 'pcr', 'qpcr', 'sti..."
4,4,Bonnie O'Connor,"'half', 'gener', 'popul', 'attend', 'screen', ..."


In [ ]:
df_tok.head()

In [15]:
ab_tok = pd.read_pickle('volume/abtokens.pkl')

In [16]:
ab_tok.head()

0    [bone, scintigraphi, wide, appli, detect, bone...
1    [cystic, fibrosi, common, autosom, recess, dis...
2    [alzheim, diseas, neurodegen, syndrom, charact...
3    [purposexeroderma, pigmentosum, group, xpd, co...
4    [class, mhc, molecul, display, peptid, cell, s...
Name: ab_toks, dtype: object

In [20]:
df_tok['ab_toks'] = ab_tok

In [28]:
df_tok = df_tok.drop(['index'], axis=1)

In [33]:
df_tok['ab_toks'] = df_tok['ab_toks'].apply(lambda x: str(x))

In [35]:
df_tok.to_sql('ed_concat_abtoks', conn, if_exists="replace")

In [7]:
df = pd.read_sql('SELECT * from ed_concat_abtoks', conn)

In [32]:
df = df.drop(['index'],axis=1)

In [18]:
df.ab_toks = df.ab_toks.apply(lambda x: x.replace("] [",", "))

In [21]:
df.editors = df.editors.apply(lambda x: x.strip('["]'))

In [23]:
#df.to_sql('ed_concat', conn, if_exists="replace")

In [14]:
df.ab_toks = df.ab_toks.apply(lambda x: x.strip("[]"))

In [33]:
df.head()

,editors,ab_toks
0,Alash'le G. Abimiku,"'objectiveto', 'evalu', 'cost', 'effect', 'qua..."
1,Alessandro D'Ausilio,"'test', 'whether', 'pre', 'assign', 'arm', 'mo..."
2,Anderson de Souza Sant'Ana,"'human', 'migrat', 'impli', 'adapt', 'new', 'e..."
3,Avi Ma'ayan,"'real', 'time', 'quantit', 'pcr', 'qpcr', 'sti..."
4,Bonnie O'Connor,"'half', 'gener', 'popul', 'attend', 'screen', ..."


In [34]:
toks = df['ab_toks'].tolist()

In [41]:
len(toks)

14918

In [37]:
tok = toks[1].split(', ')

In [38]:
tnew = []
for line in tok:
    l = line.replace("'","")
    l = l.strip('[]')
    tnew.append(l)

In [40]:
abs_ed = []

for i in range(0,len(toks)):
    det = toks[i].replace(',','')
    det = det.replace("'","")
    det = det.strip('[]')
    abs_ed.append(det)

0


In [49]:
texts = []

for i in range(0,len(abs_ed)):
    texts.append(abs_ed[i].split())

In [50]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [53]:
corpora.MmCorpus.serialize('volume2/new_models/editor_corpus.mm', corpus)
dictionary.save('volume2/new_models/editor_dictionary.dict')

In [54]:
tfidf = models.TfidfModel(corpus, normalize=True)
tfidf.save('volume2/new_models/model.tfidf')

In [59]:
start_time = time.time()
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=50)
lsi.save('volume2/new_models/editor_model.lsi')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lsi = lsi[corpus_tfidf]

--- 14.491721391677856 seconds ---


In [60]:
start_time = time.time()
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=50, passes = 4)
lda.save('volume2/new_models/editor_model.lda')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lda = lda[corpus_tfidf]

--- 264.339075088501 seconds ---


In [61]:
corpus_tfidf = tfidf[corpus]
corpus_lsi = lsi[corpus_tfidf]
corpus_lda = lda[corpus_tfidf]

In [62]:
index = similarities.MatrixSimilarity(lsi[corpus])
index.save('volume2/new_models/editor_similarity.index')

In [64]:
detokenizer = MosesDetokenizer()
file = open('volume2/processed_editor_docs.txt', 'w')

for i in range(0,len(texts)):
    detok_str = detokenizer.detokenize(texts[i], return_str=True)
    file.write("%s\n" % detok_str)

In [73]:
docs = [doc for doc in TaggedLineDocument('volume2/processed_editor_docs.txt')]

In [76]:
start_time = time.time()
model = Doc2Vec(docs, vector_size=200, window=5, min_count=3, workers=15, epochs=20)

np.save('volume2/new_models/editor_features-w2v-200.npy',model.docvecs.doctag_syn0)
model.save('volume2/new_models/editor_features-w2v-200.doc2vec')
print("--- %s seconds ---" % (time.time() - start_time))

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  after removing the cwd from sys.path.


--- 395.6190092563629 seconds ---
